<a href="https://colab.research.google.com/github/tlsdmswn01/Text_mining_project/blob/main/Dart_Doc2vec/Dart%EA%B3%B5%EC%8B%9C%EC%9E%90%EB%A3%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install dart-fss
!pip install opendartreader

In [ ]:
#dart api 인증키 : c585539e39b9f04f5aec7f95223bf660c253a73c
import OpenDartReader
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [ ]:
#pip install --upgrade opendartreader

In [ ]:
import OpenDartReader

### 0. 객체 생성 ###
# 객체 생성 (API KEY 지정)
api_key = 'c585539e39b9f04f5aec7f95223bf660c253a73c'

dart = OpenDartReader(api_key)


In [ ]:

report = dart.list('삼성전자', start='2023', kind='A', final=False)
report_nm =report[report['report_nm'].str.contains('사업보고서')]
report_num=report_nm['rcept_no']
report_num
for j in report_num:
    sub_report=dart.sub_docs(j)
    report_url =sub_report[sub_report['title'].str.contains('사업의 개요')]
    report_url2=report_url['url']
for k in report_url2:
    url = k
    soup = BeautifulSoup(requests.get(url).text)

text=soup.text

In [ ]:
# 불용어 사전
stop_words=pd.read_csv('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/웹 크롤링 코드 및 데이터/stopwords.txt',sep='\t')

In [ ]:
from konlpy.tag import Okt # komoran, hannanum, kkma, mecab
import re
import nltk
nltk.download('wordnet') #Lemmatization - 표제어 추출(통일화)
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer()
okt=Okt()
# 토큰화하기
def preprocessing(s):
    try:
    #[]제거
        s = re.sub(pattern='\[[^)]*\]\n+', repl='', string=s)

    # 특수문자 제거
        s = re.sub('[^a-zA-Z가-힣\s]', repl='', string=s)
    #원하는 불용어 제거하기
        for j in stop_words['가']:
            s = re.sub(j, repl='', string=s)
    except:
        print(s)


    # 토큰화
    s_token=okt.morphs(s,norm=True)


    # 글자 1개만 있으면 제외하기
    word_list = []
    for word in s_token:
        if len(word) !=1:
            word_list.append(wl.lemmatize(word))

    return word_list

In [ ]:
cor_text=preprocessing(text)
del cor_text[0]
del cor_text[2]

import pickle
with open("삼성전자 사업개요.pkl","wb") as f:
    pickle.dump(samsung_text, f)

In [ ]:
cor_list=pd.read_excel('/content/drive/Shareddrives/텍스트 마이닝 프로젝트/웹 크롤링 코드 및 데이터/상장법인목록.xlsx')

In [ ]:
cor_list.info()

In [ ]:
cor_list['회사명'][0]

In [ ]:
from tqdm import tqdm
import time


In [ ]:
document=[]
company_name=cor_list['회사명'][1000:]
for i in tqdm(company_name):
    try:
        report = dart.list(i, start='2023', kind='A', final=False)
        report_nm =report[report['report_nm'].str.contains('사업보고서')]
        report_num=report_nm['rcept_no']
        report_num
        for j in report_num:
            sub_report=dart.sub_docs(j)
            report_url =sub_report[sub_report['title'].str.contains('사업의 개요')]
            report_url2=report_url['url']
        for k in report_url2:
            url = k
            soup = BeautifulSoup(requests.get(url).text)
            document.append(soup.text)
    except:
        document.append('na')
document

In [ ]:
company_name = company_name.reset_index(drop=True)

print(len(company_name))
print(len(document))

a = 0
for i in range(len(document)):
    if str(document[i]) == 'na':
        a += 1
print(a)

In [ ]:
document[0]
# company_name[0]

In [ ]:
df=pd.DataFrame({'company1':company_name,'document1':document})
df

In [ ]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm